In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from scipy.special import logit, expit
import string
import re    #for regex
import nltk
from nltk.corpus import stopwords
import spacy
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize, TweetTokenizer   
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import collections
import re
import sys
import pickle
import spacy
import enchant
import gensim
import operator
import sys, datetime

/usr/local/lib/python3.5/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [19]:
from collections import Counter

In [3]:
DATA_PATH = "../data/"
sid = SentimentIntensityAnalyzer()
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
d = enchant.Dict("en_US")
nlp = spacy.load('en')
eng_stopwords = set(stopwords.words("english"))

In [4]:
train = pd.read_csv(DATA_PATH + 'raw/train.csv').fillna(' ')
test = pd.read_csv(DATA_PATH + 'raw/test.csv').fillna(' ')

In [5]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [6]:
# Separate toxic comments from clean ones:
train_toxic = train[(train.toxic == 1) | (train.obscene == 1) | (train.threat == 1) | (train.insult == 1) | (train.identity_hate == 1)]
train_toxic.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0


In [7]:
words_ethnicity = pickle.load(open("../ling_src/words_ethnicity.p", 'rb'))

In [29]:
def count_words_ethnicity(text):
    cntr = Counter(text.lower().split())
    return np.sum([cntr[word.lower()] for word in words_ethnicity])

In [25]:
words_ethnicity = list(set(words_ethnicity))

In [32]:
train_toxic['words_ethnicity'] = train_toxic['comment_text'].apply(count_words_ethnicity)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [33]:
words_gender = pickle.load(open("../ling_src/words_gender.p", 'rb'))
words_gender = list(set(words_gender))
def count_words_gender(text):
    cntr = Counter(text.lower().split())
    return np.sum([cntr[word.lower()] for word in words_gender])

In [37]:
train_toxic['words_gender'] = train_toxic['comment_text'].apply(count_words_gender)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [38]:
words_class = pickle.load(open("../ling_src/words_class.p", 'rb'))
words_class = list(set(words_class))
def count_words_class(text):
    cntr = Counter(text.lower().split())
    return np.sum([cntr[word.lower()] for word in words_class])

In [39]:
train_toxic['words_class'] = train_toxic['comment_text'].apply(count_words_class)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [40]:
words_nationality = pickle.load(open("../ling_src/words_nationality.p", 'rb'))
words_nationality = list(set(words_nationality))
def count_words_nationality(text):
    cntr = Counter(text.lower().split())
    return np.sum([cntr[word.lower()] for word in words_nationality])

In [41]:
train_toxic['words_nationality'] = train_toxic['comment_text'].apply(count_words_nationality)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [42]:
words_religion = pickle.load(open("../ling_src/words_religion.p", 'rb'))
words_religion = list(set(words_religion))
def count_words_religion(text):
    cntr = Counter(text.lower().split())
    return np.sum([cntr[word.lower()] for word in words_religion])

In [43]:
train_toxic['words_religion'] = train_toxic['comment_text'].apply(count_words_religion)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [48]:
train_toxic['words_gender'].argmax()

27148

In [50]:
train_toxic.loc[27148]['comment_text']

'DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YOU U CUNT DAMN YO

In [46]:
train_toxic[['words_religion', 'words_class',  'words_nationality', 'words_gender', 'words_ethnicity']].describe()

,words_religion,words_class,words_nationality,words_gender,words_ethnicity
count,16225.000000,16225.000000,16225.000000,16225.00000,16225.000000
mean,0.000740,0.005116,0.013313,0.15359,0.013313
std,0.027186,0.084036,0.563253,4.01740,0.563253
min,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.000000,0.00000,0.000000
50%,0.000000,0.000000,0.000000,0.00000,0.000000
75%,0.000000,0.000000,0.000000,0.00000,0.000000
max,1.000000,5.000000,70.000000,312.00000,70.000000


In [51]:
train['words_gender'] = train['comment_text'].apply(count_words_gender)
train['words_religion'] = train['comment_text'].apply(count_words_religion)
train['words_nationality'] = train['comment_text'].apply(count_words_nationality)
train['words_class'] = train['comment_text'].apply(count_words_class)
train['words_ethnicity'] = train['comment_text'].apply(count_words_ethnicity)
train.to_csv(DATA_PATH + 'preprocessed/train_hatebase.csv')

In [52]:
test['words_gender'] = test['comment_text'].apply(count_words_gender)
test['words_religion'] = test['comment_text'].apply(count_words_religion)
test['words_nationality'] = test['comment_text'].apply(count_words_nationality)
test['words_class'] = test['comment_text'].apply(count_words_class)
test['words_ethnicity'] = test['comment_text'].apply(count_words_ethnicity)
test.to_csv(DATA_PATH + 'preprocessed/test_hatebase.csv')